In [1]:
!pip install tqdm

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2

import time
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical


# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 2

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Saving vectors of label - 'cipi': 100%|█████████████████████████████████████████████| 142/142 [00:00<00:00, 246.95it/s]


In [3]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sv,sr = soundfile.read('../Ring09.wav')
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    y_pred = model.predict(sample_reshaped)
    y_max = np.max(y_pred)
    ypred = np.argmax(y_pred)
    if y_max>0.9 and ypred==1:
        print("Predicted:" ,get_labels()[0][ypred],y_max, ypred)
        sounddevice.play(sv,samplerate=16000)
    return get_labels()[0][ypred]

# Predicts one sample
def predicte(audiodata, model):
    sv,sr = soundfile.read('../Ring09.wav')
    
    audiodata = np.array(audiodata).flatten()
    sample = au2mfcc(audiodata)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    y_pred = model.predict(sample_reshaped)
    y_max = np.max(y_pred)
    ypred = np.argmax(y_pred)
    if y_max>0.9:
        print("Predicted:" ,get_labels()[0][ypred], y_max)
        sounddevice.play(sv,samplerate=16000)
    return get_labels()[0][ypred]


# Building The Model Then Training it

In [4]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(X_test, y_test_hot))

Train on 177 samples, validate on 119 samples
Epoch 1/50
 - 0s - loss: 1.4813 - acc: 0.5141 - val_loss: 0.6932 - val_acc: 0.4370
Epoch 2/50
 - 0s - loss: 0.6858 - acc: 0.5424 - val_loss: 0.6929 - val_acc: 0.4370
Epoch 3/50
 - 0s - loss: 0.6944 - acc: 0.5424 - val_loss: 0.6918 - val_acc: 0.4370
Epoch 4/50
 - 0s - loss: 0.7610 - acc: 0.5593 - val_loss: 0.6931 - val_acc: 0.7311
Epoch 5/50
 - 0s - loss: 0.6929 - acc: 0.5254 - val_loss: 0.6931 - val_acc: 0.7311
Epoch 6/50
 - 0s - loss: 0.6907 - acc: 0.6215 - val_loss: 0.6931 - val_acc: 0.7563
Epoch 7/50
 - 0s - loss: 0.6952 - acc: 0.6723 - val_loss: 0.6931 - val_acc: 0.6723
Epoch 8/50
 - 0s - loss: 0.6906 - acc: 0.6271 - val_loss: 0.6931 - val_acc: 0.5294
Epoch 9/50
 - 0s - loss: 0.6961 - acc: 0.5819 - val_loss: 0.6931 - val_acc: 0.8151
Epoch 10/50
 - 0s - loss: 0.6898 - acc: 0.6667 - val_loss: 0.6931 - val_acc: 0.9160
Epoch 11/50
 - 0s - loss: 0.6944 - acc: 0.5932 - val_loss: 0.6931 - val_acc: 0.8655
Epoch 12/50
 - 0s - loss: 0.6951 - acc:

## Prediction

In [5]:
#model.save("chippy_v1.model")

In [6]:
import keras
model = keras.models.load_model("chippy_v1.model")

In [11]:
import soundfile
import itertools
import time
import librosa
import sounddevice

DURATION=20
gain=10
range=[100,2000]
high=2000
low=100
screenwidth=79

def record(length=1, reclength=1, filename=None, thres=0):
    """ 
    Merekam suara secara stream dan metode callback
    """

    global cumulated_status, end_count, start_count, recording, magnitudo, audiodata
    end_count=False
    start_count = 0
    recording=False
    magnitudo=[]
    audiodata=[]
    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / screenwidth
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()

        def callback(indata, frames, time, status):
            global cumulated_status, audiodata, magnitudo, end_count, start_count, recording, model
            
            
            cumulated_status |= status
            if any(indata):
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize

                rms = librosa.feature.rmse(S=indata)
                rms = int(rms*32768)
                start_count += 1
                if rms>=thres:
                    if not recording :                    #and not end_count
                        print("Start record")
                        recording = True
                        start_count = 0
                        
                        
                if recording:
                    audiodata.extend(itertools.chain(indata.tolist()))
                    magnitudo.append(magnitude)
                    if start_count == int(samplerate / (samplerate * DURATION / 1000)):
                        print("End record")
                        start_count=0
                        end_count=True
                        recording=False
                        try:
                            soundfile.write("temp.wav",audiodata,16000)
                            predict("temp.wav", model=model)
                        except:
                            pass
                        audiodata=[]



        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            while True:
                #response = input()
                #if response in ('', 'q', 'Q'):
                time.sleep(length)
                break
            if filename!=None: soundfile.write(filename,audiodata,16000)

        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)


In [9]:
predict('temp.wav', model=model)

Predicted: cipi 0.9999989 1


'cipi'

In [10]:
record(length=25,filename="record.wav",thres=1100)

Start record
End record
Predicted: cipi 0.9999808 1
Start record
End record
Start record
End record
Predicted: cipi 0.9999962 1
Start record
End record
Start record
End record
Predicted: cipi 0.99997807 1
Start record
End record
Start record
End record
Start record
End record
Start record
End record
Start record
End record
Predicted: cipi 0.9999635 1
Start record
End record
Start record
End record
Predicted: cipi 0.99990034 1
Start record
End record
Start record
End record
Predicted: cipi 0.9999802 1
Start record
End record


In [ ]:
predict("temp.wav", model=model)

In [ ]:
import sounddevice

In [ ]:
soundfile.read("../Ring09.wav")

In [ ]:
import librosa

In [ ]:
audiodata = b''.join(np.array(audiodata).flatten())

In [ ]:
from preprocess import *

In [ ]:
aaa = np.array(audiodata).flatten()

In [ ]:
sv, sr = soundfile.read("../Ring09.wav")
sounddevice.play(sv,samplerate=16000)